- title: Looking for Patterns in City Names & Interactive Plotting Jupyter
- author: Jack McKew
- date: 2019-08-16 06:30
- category: Data Science
- tags: python, datascience
- slug: looking-for-patterns-in-city-names-interactive-plotting-jupyter

# Looking for patterns in city names

Recently, I was travelling around New Zealand, and noticed in the maori language they use letters back to back a lot like in the original Maori name for Stratford ("whakaahurangi").

So as any normal person does, I thought, well what town has the most repeated letters, and the idea for this blog post was born.

Firstly, we'd have to find a dataset of all the town names, and found the database for all world cities hosted on Kaggle here: https://www.kaggle.com/max-mind/world-cities-database.


In [1]:
import pandas as pd
import collections
from collections import OrderedDict
import operator
# import matplotlib.pyplot as plt
import numpy as np
import math
from bokeh.plotting import figure, show, output_notebook,output_file, save
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.models import ColumnDataSource, HoverTool
from pyproj import Proj, transform
from bokeh.resources import CDN
from bokeh.embed import file_html

## Get the data!

In [2]:
# data source https://www.kaggle.com/max-mind/world-cities-database
cities_df = pd.read_csv('./data/worldcitiespop.csv', header=0, sep=',', quotechar='"')
cities_df = cities_df[cities_df['Country'] == "nz"]
display(cities_df)

C:\Users\jackm\Anaconda3\envs\bokeh-letters\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Country,City,AccentCity,Region,Population,Latitude,Longitude
2047989,nz,abbotsford,Abbotsford,G2,NaN,-45.883333,170.416667
2047990,nz,adams flat,Adams Flat,G3,NaN,-46.116667,169.833333
2047991,nz,addington,Addington,E9,NaN,-43.550000,172.616667
2047992,nz,admiralty bay,Admiralty Bay,F7,NaN,-40.950000,173.916667
2047993,nz,ahaura,Ahaura,G3,NaN,-42.350000,171.533333
2047994,nz,ahikiwi,Ahikiwi,E7,NaN,-35.783333,173.733333
2047995,nz,ahimia,Ahimia,E7,NaN,-36.850000,175.483333
2047996,nz,ahipara,Ahipara,E7,1093.0,-35.166667,173.166667
2047997,nz,ahititi,Ahititi,F1,NaN,-38.433333,178.050000
2047998,nz,ahuiti,Ahuiti,G2,NaN,-39.633333,175.000000


After inspecting the data of this dataset, we're able to filter out to look at just New Zealand with the prefix of "nz" in the Country column. It must be noted that this dataset represents the names of the towns currently, and not the original Maori names, more on this will be covered in a later post. Now we want to extract the town names
out of the dataframe with the ones we want to analyse. For ease later on, we will extract this as a dictionary,
such that we can assign the value of each to the count of each letter.

In [3]:
nz_cities = cities_df[cities_df['Country'] == "nz"]['AccentCity'].tolist()
nz_dict = { i : 0 for i in nz_cities }
display(nz_dict)

{'Abbotsford': 0,
 'Adams Flat': 0,
 'Addington': 0,
 'Admiralty Bay': 0,
 'Ahaura': 0,
 'Ahikiwi': 0,
 'Ahimia': 0,
 'Ahipara': 0,
 'Ahititi': 0,
 'Ahuiti': 0,
 'Ahurangi': 0,
 'Ahuriri': 0,
 'Ahuriri Village': 0,
 'Ahuroa': 0,
 'Aickens': 0,
 'Aka Aka': 0,
 'Akatore': 0,
 'Akerama': 0,
 'Akitio': 0,
 'Albany': 0,
 'Albert Road': 0,
 'Albert Town': 0,
 'Albury': 0,
 'Alford Forest': 0,
 'Alfredton': 0,
 'Algies Bay': 0,
 'Allandale': 0,
 'Allanton': 0,
 'Allerton': 0,
 'Alma': 0,
 'Altimarloch': 0,
 'Altimarlock': 0,
 'Alton': 0,
 'Amberley': 0,
 'Amodeo': 0,
 'Amodeo Bay': 0,
 'Anama': 0,
 'Annat': 0,
 'Aohanga': 0,
 'Aokautere': 0,
 'Aongatete': 0,
 'Aorere': 0,
 'Aotea': 0,
 'Aotuhia': 0,
 'Aparima': 0,
 'Apata': 0,
 'Apiti': 0,
 'Appleby': 0,
 'Arahura': 0,
 'Arai Point': 0,
 'Aramiro': 0,
 'Aramoana': 0,
 'Aramoho': 0,
 'Aranga': 0,
 'Aranui': 0,
 'Arapae': 0,
 'Arapito': 0,
 'Arapohue': 0,
 'Arapuni': 0,
 'Ararata': 0,
 'Ararimu': 0,
 'Aratapu': 0,
 'Arcadia': 0,
 'Ardlussa': 0,

Now we will create an ordered dictionary with the help from the collections package which will store the values
of the count for each letter in the town name.

In [4]:
letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

lcount = dict(OrderedDict([(l, 0) for l in letters]))
display(lcount)

{'A': 0,
 'B': 0,
 'C': 0,
 'D': 0,
 'E': 0,
 'F': 0,
 'G': 0,
 'H': 0,
 'I': 0,
 'J': 0,
 'K': 0,
 'L': 0,
 'M': 0,
 'N': 0,
 'O': 0,
 'P': 0,
 'Q': 0,
 'R': 0,
 'S': 0,
 'T': 0,
 'U': 0,
 'V': 0,
 'W': 0,
 'X': 0,
 'Y': 0,
 'Z': 0}

Now it's time for the data crunch. To count how many times a letter repeats in a town name we follow these steps:

- we create a for loop, to loop through all the city names in the table,
- initialise an ordered dictionary similar to above for each city in the value field of that town's dictionary entry
- loop through each letter in the town name
- check if the letter appears in our letter dictionary (mainly to not count spaces),
- Then if the letter does appear, increment the value for that letter by 1

This results in a dictionary for each town name, with the count of repeated letters.

In [5]:
for city in nz_cities:
    nz_dict[city] = dict(OrderedDict([(l, 0) for l in letters]))
    city_dict = nz_dict[city]
    for c in city:
        if c.upper() in letters:
            city_dict[c.upper()] += 1

display(nz_dict) 

{'Abbotsford': {'A': 1,
  'B': 2,
  'C': 0,
  'D': 1,
  'E': 0,
  'F': 1,
  'G': 0,
  'H': 0,
  'I': 0,
  'J': 0,
  'K': 0,
  'L': 0,
  'M': 0,
  'N': 0,
  'O': 2,
  'P': 0,
  'Q': 0,
  'R': 1,
  'S': 1,
  'T': 1,
  'U': 0,
  'V': 0,
  'W': 0,
  'X': 0,
  'Y': 0,
  'Z': 0},
 'Adams Flat': {'A': 3,
  'B': 0,
  'C': 0,
  'D': 1,
  'E': 0,
  'F': 1,
  'G': 0,
  'H': 0,
  'I': 0,
  'J': 0,
  'K': 0,
  'L': 1,
  'M': 1,
  'N': 0,
  'O': 0,
  'P': 0,
  'Q': 0,
  'R': 0,
  'S': 1,
  'T': 1,
  'U': 0,
  'V': 0,
  'W': 0,
  'X': 0,
  'Y': 0,
  'Z': 0},
 'Addington': {'A': 1,
  'B': 0,
  'C': 0,
  'D': 2,
  'E': 0,
  'F': 0,
  'G': 1,
  'H': 0,
  'I': 1,
  'J': 0,
  'K': 0,
  'L': 0,
  'M': 0,
  'N': 2,
  'O': 1,
  'P': 0,
  'Q': 0,
  'R': 0,
  'S': 0,
  'T': 1,
  'U': 0,
  'V': 0,
  'W': 0,
  'X': 0,
  'Y': 0,
  'Z': 0},
 'Admiralty Bay': {'A': 3,
  'B': 1,
  'C': 0,
  'D': 1,
  'E': 0,
  'F': 0,
  'G': 0,
  'H': 0,
  'I': 1,
  'J': 0,
  'K': 0,
  'L': 1,
  'M': 1,
  'N': 0,
  'O': 0,
  'P': 0,

Hooray! Now we have all the data we need broken down and ready for analysis. To help ease the analysis and make it more readable for a human, we convert from our nested dictionaries to a pandas dataframe and transpose it such that we
have the town name as the index, the letters as the column and the count of that letter as the values.

In [6]:
total_df = pd.DataFrame.from_dict(nz_dict)
total_df = total_df.T
display(total_df)

,A,B,C,D,E,F,G,H,I,J,...,Q,R,S,T,U,V,W,X,Y,Z
Abbotsford,1,2,0,1,0,1,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0
Adams Flat,3,0,0,1,0,1,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
Addington,1,0,0,2,0,0,1,0,1,0,...,0,0,0,1,0,0,0,0,0,0
Admiralty Bay,3,1,0,1,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,2,0
Ahaura,3,0,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,0,0,0
Ahikiwi,1,0,0,0,0,0,0,1,3,0,...,0,0,0,0,0,0,1,0,0,0
Ahimia,2,0,0,0,0,0,0,1,2,0,...,0,0,0,0,0,0,0,0,0,0
Ahipara,3,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,0,0,0
Ahititi,1,0,0,0,0,0,0,1,3,0,...,0,0,0,2,0,0,0,0,0,0
Ahuiti,1,0,0,0,0,0,0,1,2,0,...,0,0,0,1,1,0,0,0,0,0


Now we want to find which of these names have the maximum count for any particular letter and store it in a summary dataframe. It is to be noted that we could use the pivot function with aggregate types, however, I have not figured a nice way to do this yet. If you do know a nicer way to determine this, please let me know.

In [7]:
summary_df = pd.DataFrame()
scale = 1
summary_df['City_Name'] = total_df.idxmax()
summary_df['Count'] = total_df.loc[total_df.idxmax()].max()
display(summary_df)

,City_Name,Count
A,Kaingapai Hakataramea Station,9
B,Abbotsford,2
C,Christchurch,3
D,Edendale Town District,3
E,Earnscleugh Settlement,5
F,Flagstaff,3
G,Kyeburn Diggings,3
H,Christchurch,3
I,Kihikihi Town District,6
J,Clarks Junction,1


Now by using the equivalent of an index-match in excel which you can read more about here (https://towardsdatascience.com/name-your-favorite-excel-function-and-ill-teach-you-its-pandas-equivalent-7ee4400ada9f). Admittedly, we could've made the join earlier, but since I use index-match so often in Excel, I wanted to learn how to do the same in pandas. This is achieved by using the map function (which is the equivalent of the index), but by using the index of another dataframe as the argument (the match function), we can rejoin the dataset by matching the city name from our original dataset.

In [8]:
summary_df['Latitude'] = summary_df['City_Name'].map(cities_df.set_index(['AccentCity'])['Latitude'].to_dict()) * scale
summary_df['Longitude'] = summary_df['City_Name'].map(cities_df.set_index(['AccentCity'])['Longitude'].to_dict()) * scale
display(summary_df)

,City_Name,Count,Latitude,Longitude
A,Kaingapai Hakataramea Station,9,-44.600000,170.566667
B,Abbotsford,2,-45.883333,170.416667
C,Christchurch,3,-43.533333,172.633333
D,Edendale Town District,3,-46.316667,168.783333
E,Earnscleugh Settlement,5,-45.216667,169.316667
F,Flagstaff,3,-45.833333,170.483333
G,Kyeburn Diggings,3,-45.000000,170.283333
H,Christchurch,3,-43.533333,172.633333
I,Kihikihi Town District,6,-38.033333,175.350000
J,Clarks Junction,1,-45.733333,170.050000


Now we have a dataframe that contains:
- an index of the letters,
- the town name with the most repeated letters,
- the count of the letters within the name,
- the longitude and latitude of the town

For plotting with bokeh on a basemap, we need to convert from longitude & latitude to easting and northing. To do this we use the pyproj package to make this very simple.

In [9]:
def LongLat_to_EN(long, lat):
    try:
      easting, northing = transform(
        Proj(init='epsg:4326'), Proj(init='epsg:3857'), long, lat)
      return easting, northing
    except:
      return None, None

This function can be used to generate the easting and northing for every town from it's longitude & latitude and add it to the dataframe.

In [10]:
summary_df['E'], summary_df['N'] = zip(*summary_df.apply(lambda x: LongLat_to_EN(x['Longitude'], x['Latitude']), axis=1))

display(summary_df)

,City_Name,Count,Latitude,Longitude,E,N
A,Kaingapai Hakataramea Station,9,-44.600000,170.566667,1.898739e+07,-5.558768e+06
B,Abbotsford,2,-45.883333,170.416667,1.897070e+07,-5.761673e+06
C,Christchurch,3,-43.533333,172.633333,1.921745e+07,-5.393506e+06
D,Edendale Town District,3,-46.316667,168.783333,1.878887e+07,-5.831241e+06
E,Earnscleugh Settlement,5,-45.216667,169.316667,1.884825e+07,-5.655696e+06
F,Flagstaff,3,-45.833333,170.483333,1.897812e+07,-5.753681e+06
G,Kyeburn Diggings,3,-45.000000,170.283333,1.895585e+07,-5.621521e+06
H,Christchurch,3,-43.533333,172.633333,1.921745e+07,-5.393506e+06
I,Kihikihi Town District,6,-38.033333,175.350000,1.951987e+07,-4.584136e+06
J,Clarks Junction,1,-45.733333,170.050000,1.892988e+07,-5.737718e+06


Finally, it's time to plot our findings on a map. Before we initialise the map in Bokeh [LINK BOKEH], for most plots, data tables and more in Bokeh, we need to put it in the ColumnDataSource form. We also initialise the interactivity when the user hovers over the data points on the plot.

In [11]:
source = ColumnDataSource(data=dict(
                        longitude=list(summary_df['E']), 
                        latitude=list(summary_df['N']),
                        sizes=list(summary_df['Count']*3),
                        lettercount = list(summary_df['Count']),
                        city_name=list(summary_df['City_Name']),
                        letters = list(summary_df.index)))

hover = HoverTool(tooltips=[
    ("Repeated Letter" , "@letters"),
    ("City Name", "@city_name"),
    ("Count","@lettercount")
    
])

Finally time for the plot! Now admittedly, I haven't found an easy way to find the limits of the graph, so this was made with a lot of trial and error (If you know a better way, please let me know!).

In [12]:
p = figure(x_range=(20000000,17900000), y_range=(-6000000,-4000000),x_axis_type="mercator", y_axis_type="mercator",tools=[hover, 'wheel_zoom','save'])
p.add_tile(CARTODBPOSITRON)
p.circle(x='longitude',
         y='latitude', 
         size='sizes',
         source=source,
         line_color="#FF0000", 
         fill_color="#FF0000",
         fill_alpha=0.05)
output_notebook()
show(p)

Loading BokehJS ...

In [13]:
# output_file("NZ_City_Letter_Analysis.html")
# save(p)
# html = file_html(p, CDN, "NZ_City_Letter_Analysis")